# Task 1 - Exploratory data analysis
## Group 8
#### Godun Alina, Börtlein Lorenz, Kodar Mark-Eerik, Car Franko
### Algorithm - P3C
### Dataset - ENZYMES


### P3C (Projected Clustering via Cluster Cores)

P3C has the following properties:
>- Effectively discovers the projected clusters in the data while being remarkable robust to the only parameter that it takes as input. The setting of this parameter requires little prior knowledge about the data and, in contrast to all previous approaches, there is no need to provide the number of projected clusters as input, since our algorithm can discover, under very general conditions, the true number of projected clusters.
>- Effectively discovers very low-dimensional projected clusters embedded in high-dimensional spaces.
>- Effectively discovers clusters with varying orientation in their relevant subspaces.
>- Scalable with respect to large data sets and high number of dimensions. 

The algorithm is comprised of the following steps:
>- Regions corresponding to projections of clusters onto single attributes are computed.
>- Cluster cores are identified by spatial areas that (1) are described by combination of the detected regions and (2) contain an unexpectedly large number of points.
>- Cluster cores are iteratively refined into projected clusters
>- Outliers are identified and the relevant attributes for each cluster are determined.

### Introduction

We chose the enyzmes dataset as one of our team members had prior experience with the field and it seemed interesting to the other participants as well. After that we proceeded with the algorithm selection and thought that P3C might suit our dataset due to the fact that it would be scalable with high number of dimensions and would be relatively easy to implement as it required tuning of only one parameter. In addition to that, the original paper stated that it is faster than some of the other algorithms we could choose 

In reality the latter is not true as there are more parameters that influence the performance of the algorithm: computation for the number of bins, degrees of freedom for noise prediction.
In addition to that we had two team members leave the project one of which was the member with the knowledge of enzymes. This meant that further development was influenced by that and it made the evaluation of our results harder. 

### P3C implementation steps

The implementation has the following steps:
- Projections of true $p$-signatures
    - <u>Idea</u>: For each attribute compute the intervals that match or approximate well the projections of true $p$-signatures onto that attribute.
    - Identify attributes with uniform distribution and for the non-uniform attributes, to identify intervals with unusual high support using Chi-square goodness-of-fit test.
    - Each attribute is divided into same number of equi-sized bins. For every bin in every attribute, its support is computed.
    - On the attributes deemed non-uniform, the bin with the largest support is marked. The remaining un-marked bins are tested again using the Chi-square test for uniform distribution. If the Chi-square test indicates that the un-marked bins "look" uniform, then we stop. Otherwise, the bin with the second-largest support is marked. Repeat testing for remaining un-marked bins for the uniform distribution and marking bins in decreasing order of support, until the current set of un-marked bins satisifies the test. 
    - Compute intervals by merging adjacent marked bins. This marking process of bins is linear in the number of bins. 
- Finding the cluster cores
    - <u>Idea</u>: Determining which calculated $p$-signatures do in fact represent the projected clusters.
    - Compute how many points are expected to belong to the support set to a specific support set in the case when they are not part of the true $t$-signature.
    - Compute the expected support of the signatures.
    - Check whether a support set belongs to the true $t$-signature by comparing the expected support to the actual support with the help of Poission probability density function. 
    - The results of that are considered to be the cluster cores.
- Computing the projected clusters
    - <u>Idea</u>: The support sets of the cluster cores found in the previous point may not necessarily contain all and only the points of the projected clusters that the cluster cores approximate, depending on the accuracy of the intervals computed in the first step. Refine found $k$ cluster cores into $k$ projected clusters. This is performed in a subscape of (reduced) dimensionality $d'$ of the original $d$-dimensional data, containing all attributes that were deemed non-uniform.
    - Describe the membership of data points to cluster cores through a fuzzy membership matrix $M=(m_{il})_{i=1,n,l,=1,k}$, where $m_{il}$ denotes the membership of object $i$ to cluster core $l$; it is defined as follows: $m_{il}=0$ if data point $i$ does not belong to the support set of any cluster core; $m_{il}$ is equal to the fraction of cluster cores that contain data point $i$ in their support set, if $i$ is in the support set of cluster core $l$. 
    - Compute the probability of a data point belonging to each projected cluster using Expectation Maximization algorithm which is initialized with the fuzzy membership matrix $M$. 
    - Assign points that have value 0 in the fuzzy membership matrix to one of the cluster cores with the shortest Mahalanobis distance to the cluster mean.
    - EM returns the matrix of probabilities that gives for each data point its probability of belonging to each projected cluster and assign each data point to the most probable cluster.
- Detect outliers
    - <u>Idea</u>: Find points that should not be a part of any cluster.
    - Use standard technique for multivariate outlier detection: The Mahalanobis distances between data points and the means of the projected clusters to which they belong are compared to the critical value of the Chi-square distribution with $d'$ degrees of freedom at a confidence level of $\alpha=0.001$.
    - Data points with Mahalanobis distances larger than this critical value are declared outliers. 
- Detect relevant attributes
    - <u>Idea</u>: Determine the relevant attributes of each projected cluster based on the cluster members.
    - The relevant attributes of a projected cluster include the attributes of the intervals that make up the $p$-signature of the cluster core based on which this cluster has been computed. 
    - Test each projected cluster using the Chi-square test whether its members are uniformly distributed in the attributes initially deemed uniform. 
    - Members of a projected cluster that are not uniformly distributed in one of the attributes initially considered uniform are considered to be relevant for the projected cluster.
    - The $p$-signatures of projected clusters are refined by computing the smallest interval that the cluster members project to for each relevant attribute. 


### Our implementation results
After implementing the algorithm we evaluated it by comparing its clustering results with the results of the ELKI implementation. For the implementation comparison, we have used two datasets provided by ELKI: mouse and vary density. To verify different steps of the algorithm during the implementation, we have also generated a custom synthetic dataset consisting of multiple well-separated uniformly distributed clusters.

As we can see, PC3 does not work well for both mouse and vary density datasets in both our and ELKI’s implementations. The reason for this is that the algorithm heavily relies on the bins provided in the first step, and in both datasets, the points are not well separated in either of the dimensions, so PC3 finds only one big bin in each of the dimensions.

#### PC3 ELKI result for mouse dataset

<img src="ELKI_MOUSE_DS.PNG">

#### PC3 our implementation result for mouse dataset
<img src="ours_mouse_DS.PNG">

#### PC3 ELKI result for vary dataset
<img src="ELKI_vary_DS.PNG">

#### PC3 our implementation result for vary dataset
<img src="ours_vary_DS.PNG">

The algorithm works much better on the custom dataset, where the clusters are well separated in each of the dimensions, and subsequent cluster core search successfully identifies bins intersections in multiple dimensions.

#### PC3 ELKI result for synthetic dataset
<img src="ELKI_synthetic_DS.PNG">

#### PC3 our implementation result for synthetic dataset
<img src="ours_synthetic_DS.PNG">